In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from torchvision import datasets, transforms 
import torch 

# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0,5, ), (0.5, )), ]) ??? "," ??? NGU THE
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, ), (0.5, )), ])

train = datasets.FashionMNIST('FashionMNIST', train=True, download=True, transform=transform)
training_data = torch.utils.data.DataLoader(train, shuffle=True, batch_size=64)

test = datasets.FashionMNIST('FashionMNIST', train=False, download=True, transform=transform)
testing_data = torch.utils.data.DataLoader(test, shuffle=True, batch_size=64)

In [19]:
import torch.nn as nn 
import torch.nn.functional as F

class Net(nn.Module):

  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784, 256)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 64)
    self.fc4 = nn.Linear(64, 10)

    self.dropout = nn.Dropout(p=0.2)
    
  def forward(self, x):
    x = self.dropout(F.relu(self.fc1(x)))
    x = self.dropout(F.relu(self.fc2(x)))
    x = self.dropout(F.relu(self.fc3(x)))
    x = F.log_softmax(self.fc4(x), dim=1)

    return x
    
model = Net()
model1 = Net()
model

Net(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [34]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters(), lr=0.003)
criterion = nn.NLLLoss()

epochs = 5

for e in range(epochs):
  running_loss = 0

  for images, labels in training_data:
  
    optimizer.zero_grad()

    outputs = model(images.view(-1, 784))
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()

  else:

    correct = 0
    test_loss = 0 
    accuracy = 0

    with torch.no_grad():
      for images, labels in testing_data:

        outputs = model(images.view(-1, 784))
        test_loss += criterion(outputs, labels)

        top_p, top_class = outputs.topk(1, dim=1)
        equal = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equal.type(torch.float))
      
    print("EPOCH : {}/{}..".format(e+1, epochs))
    print("Training loss : {:.3f}".format(running_loss/len(training_data)))
    print("Test loss: %.3f" %(test_loss/len(testing_data)))
    print("Accuracy : %.2f %%" %(100*accuracy/len(testing_data)))




EPOCH : 1/5..
Training loss : 0.408
Test loss: 0.471
Accuracy : 84.42 %
EPOCH : 2/5..
Training loss : 0.395
Test loss: 0.473
Accuracy : 84.28 %
EPOCH : 3/5..
Training loss : 0.388
Test loss: 0.457
Accuracy : 84.33 %
EPOCH : 4/5..
Training loss : 0.387
Test loss: 0.454
Accuracy : 84.77 %
EPOCH : 5/5..
Training loss : 0.384
Test loss: 0.443
Accuracy : 85.32 %


In [0]:
torch.save(model.state_dict(), "checkpoint.pth")


In [0]:
#save weigths and bias matrices for each of our layers
state_dict = torch.load("checkpoint.pth")
# print(state_dict)

In [31]:
print(model)
model1.load_state_dict(state_dict)
print(model1)

Net(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
Net(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [0]:
model = model1 